# Training of the model
## Extra lib instalation

In [35]:
!pip install speechpy
!pip install soundfile
!pip install tables

## Lib imports

In [1]:
import numpy as np
import pandas as pd
import soundfile as sf
import scipy.io.wavfile as wav
import speechpy
import json
import os
import math
import tensorflow.keras as k
import dask.dataframe as dd
from IPython.display import display, Markdown
from time import sleep
from pprint import pprint
from multiprocessing import Queue, Process, Pool

%config IPCompleter.greedy=True

For audio processing I will use `speechpy` as it is the fastest of the well known libraries:

![speed_comp](https://camo.githubusercontent.com/1465ddaba9f99df4ff86ef800ef4598f35c12698/68747470733a2f2f696d61676573322e696d67626f782e636f6d2f64652f31302f6f4e5a776b49694b5f6f2e706e67)
> Source: [sonopy](https://github.com/MycroftAI/sonopy)

## Processing function for dataset creation

In [16]:
def df_col2numpy(df, col_names):
    ser = df.apply(lambda row: np.array([row[col] for col in col_names]).flatten(), axis=1)
    arr = np.array(ser.values.tolist())
    return arr

def df_col2series(df, col_names):
    if len(col_names) == 1:
        ser = df[col_names[0]].map(lambda cell: np.array([cell]).flatten())
    ser = df.apply(lambda row: np.array([row[col] for col in col_names]).flatten(), axis=1)
    return ser

def file2mfcc(file_name, frame_length=0.20, frame_stride=0.1, recreate=False):
    """ recreate: whether to recreate existing .npy MFCC"""
    
    dir_name = file_name[:file_name.index('blocks')]
    file_wav, file_ogg = None, None
    
#     check for existing .wav or .npy cache
    for file in os.listdir(dir_name):
#         if file.endswith('.wav'):
#             file_wav = os.path.join(dir_name, file)
#         if file.endswith('.npy') and not recreate:
#             return np.load(os.path.join(dir_name, file))
        if file.endswith('.mfcc') and not recreate:
            return pd.read_hdf((os.path.join(dir_name, file)))
            
#     if none .wav found, create it
    for file in os.listdir(dir_name):
        if file.endswith('.ogg'):
            file_ogg = os.path.join(dir_name, file)
            if not file_wav:
                data, samplerate = sf.read(file_ogg)
                file_wav = f'{file_ogg[:-4]}.wav'
                sf.write(file_wav, data, samplerate)

    fs, signal = wav.read(file_wav)
    
#     Stereo to mono
    if signal.shape[1] == 2:
        signal = (signal[:, 0] + signal[:, 1]) / 2
    else:
        signal = signal[:, 0]

    # Pre-emphasize
    signal_preemphasized = speechpy.processing.preemphasis(signal, cof=0.98)

    # Extract MFCC features
    mfcc = speechpy.feature.mfcc(signal, sampling_frequency=fs, frame_length=frame_length, 
                                 frame_stride=frame_stride, num_filters=40, fft_length=512,
                                 low_frequency=0, high_frequency=None, num_cepstral=13)
    
#     Normalize
    mfcc_cmvn = speechpy.processing.cmvnw(mfcc,win_size=301,variance_normalization=True)
    
#     Cache results and clean .wav to save space
#     np.save(f'{file_wav[:-4]}.mfcc.npy', mfcc_cmvn)
    if file_ogg:
        os.remove(file_wav)

#     Recalculate the time differences
    index = np.arange(0, (len(mfcc) - 0.5) * frame_stride, frame_stride) + frame_length
    df = pd.DataFrame(data=mfcc_cmvn, index=index).apply(np.array, axis=1)
    df.to_hdf(f'{file_wav[:-4]}.mfcc', 'mfcc', mode='w', format='fixed')
    return df

def process_cell(cell, side):
    res = cell[:, :, side, :]
    
    mx = res.max()
    mx_index = np.unravel_index(res.argmax(), res.shape)
    pred = [None for _ in range(3)]
    
    for dim in range(3):
#         pred[dim] = np.zeros(res.shape[dim] + 1)
        pred[dim] = np.zeros(res.shape[dim])
        
        if mx < 0.5:
#             pred[dim][-1] = 1
            pass
        else:
            pred[dim][mx_index[dim]] = 1
    
    if mx < 0.5:
        res = cell[:, :, (side+1) % 2, :]
    
        mx = res.max()
        mx_index = np.unravel_index(res.argmax(), res.shape)
        for dim in range(3):
            pred[dim][mx_index[dim]] = 1
        
    return pred

def change_output(df: pd.DataFrame):
    left, right = df.apply(lambda cell: process_cell(cell, 0)), df.apply(lambda cell: process_cell(cell, 1))
    
    left = pd.DataFrame(left.to_list(), columns=[f'l_dim{x}' for x in range(3)], index=left.index)
    right = pd.DataFrame(right.to_list(), columns=[f'r_dim{x}' for x in range(3)], index=right.index)
    
    return left.join(right)
    
def process_file(file_path, recreate=False):
    """ Processing needed to be done per file """
    print(f'Processing {file_path}')
    try:
        df = pd.read_pickle(file_path)
    
        # all in one serialization (99.5 % with bad metric)
        #     df['output'] = df_col2series(df, ['output'])
        df = df.join(change_output(df['output']))
        df['shifted'] = df['output'].shift(1, fill_value=[np.zeros(df['output'].iloc[0].shape)])
        
        df['times'] = df_col2series(df, ['prev', 'next'])
        df['name'] = f'{file_path}'

        mfcc = file2mfcc(file_path, recreate=recreate)
        mfcc.name = 'mfcc'
        round_index = mfcc.index.values[1] - mfcc.index.values[0]
        df.index = np.floor(df['time'] / round_index).astype(int)
        mfcc.index = (mfcc.index / round_index).astype(int)

        df = df.join(mfcc)
        df.index = df['time']
        df = df.dropna()
    except Exception as e:
        print(f'Caught Error: {e}')
        return None

    return df

def process_df(df, X_cols, y_cols):
    """ Post processing on the whole DF """
    
    # Add shifter y (predictions)
    y_cols_shifted = [f'{x}_shifted' for x in y_cols]
    shifted = df[y_cols].groupby('name').shift(1)
    df[y_cols_shifted] = shifted
    df = df.dropna()
    
    return df
    
file = '../data/Army Of The Night/blocks/Expert.pkl'
file2mfcc(file, recreate=False)
process_file(file).iloc[0]

Processing ../data/Army Of The Night/blocks/Expert.pkl


output     [[[[0. 1. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...
time                                                 3.08211
prev                                                 3.08211
next                                               0.0821053
l_dim0                                       [1.0, 0.0, 0.0]
l_dim1                                  [1.0, 0.0, 0.0, 0.0]
l_dim2         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
r_dim0                                       [1.0, 0.0, 0.0]
r_dim1                                  [1.0, 0.0, 0.0, 0.0]
r_dim2         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
shifted    [[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...
times              [3.0821052631578945, 0.08210526315789446]
name             ../data/Army Of The Night/blocks/Expert.pkl
mfcc       [0.36976948, -1.4347119, 0.909154, 0.62588304,...
Name: 3.0821052631578945, dtype: object

## Generate urls to blocks

In [3]:
def get_file_paths(path, hard_max):
    """ 
    Create a list of all pregenerated blocks files 
    Search in full subtree of :path:
    """
    
    file_paths = []
    counter = 0
    
    for root, dirs, files in os.walk(path, topdown=False):
        if counter > hard_max:
                break
        for name in files:
            if root[-6:] == 'blocks':
                print(f'#{counter:5} {root}/{name}')
                file_paths.append(os.path.join(root, name))
                
                counter += 1
                
    return file_paths

## Set which cols to use as X and Y

In [4]:
X_list = []
Y = []
HARD_MAX = 20000
path = '../data'

y_cols = [f'{side}_dim{i}' for side in 'rl' for i in range(3) ]
X_cols = ['times', 'mfcc']
# X_cols = y_cols

columns = ['name', 'time'] + X_cols + y_cols

## Create and save training data

In [ ]:
multi_core = True
if multi_core:
    # Embarrassingly parallel problem, but RAM heavy
    pool = Pool(processes=None)
    X_list = pool.map(process_file, get_file_paths(path, HARD_MAX))
    pool.close()
    pool.join()
else:
    X_list = [process_file(x) for x in get_file_paths(path, HARD_MAX)]

X_list = [x for x in X_list if x is not None]
        
print(f'Passes {len(X_list):6}/{HARD_MAX:6} hard max')
X = pd.DataFrame(pd.concat(X_list), columns=columns)
X = X.set_index(['name', 'time'])
X = process_df(X, X_cols, y_cols)

In [25]:
X.to_pickle(os.path.join(path, 'X_saved.pkl'))

## Load training data

In [ ]:
X = pd.read_pickle(os.path.join(path, 'X_saved.pkl'))
print(f'Loaded {len(X)} rows')

# Puff, please here upload it like above, but from the _internet_

In [ ]:
# X = magic()

# Training approaches

## Generator training approach

### Advantages
- Every batch can have different length
    - No crop of songs in batches needed
    - Minimal padding
    - $\Rightarrow$ model learns from songs with the context of beginning and end
    
### Disadvantages
- Less convinient then standard `model.fit(X, y, **kwargs)`
- Big memory movement overhead
    - Would be even more significant if trained on a GPU
    

## Standard training approach
- Different lengths solved by generating snippets of songs of len $N$
    - Effectively creates more versitile dataset, since songs are "starting" at different places
    
### Advantages
- No padding and no crop
    - No crop of songs in batches needed
    - Minimal padding
    - $\Rightarrow$ model learns from songs with the context of beginning and end
- All "heavy altilery" from TF can be used
    
### Disadvantages
- No differentitation between beggining and end of the song
    - $\Rightarrow$ could be solved by adding procentage column which indicates in interval $(0, 1)$ where the beat lies in the song

# Generator training approach
## Helper functions for train_generator

In [6]:
precision = 2

def pp(row):
    print('*' * 69)
    print(row)
    
def round_up(num:float, prec: int) -> int:
    return int(math.ceil((10 ** -prec) * num) / (10 ** -prec))
    
def get_len_category(song, prec: int=precision):
    return int(round_up(len(song), 2))

def get_mask(X, prec: int=precision):
    mask = X.groupby('name').apply(get_len_category)
    return mask.to_dict()

def create_batch(group, ceil_len, verbose=True):
    if verbose:
        print(f'Creating batch of ceil_len {ceil_len:6} with {len(group)} rows')
    ceil_len = int(ceil_len)
    
    batch = []
    for name, song in group.groupby('name'):
        empty_row = song.head(1).squeeze().apply(np.zeros_like)
#         print(f'{ceil_len} | {len(song)} | {empty_row}')
        df_to_add = pd.DataFrame([empty_row] * (ceil_len - len(song)))
        batch.append(pd.concat([song, df_to_add]))

    return pd.concat(batch)

# LESON: Don't forget about the NaNs!


def list2numpy(batch, col_name, groupby=('name')):
    return np.array(batch.groupby(list(groupby))[col_name].apply(list).to_list())

## Show bucketing results

In [7]:
grouped = X.groupby(get_mask(X), level=0)

adjust = 6
stats = []
print(f'{"from":>{adjust}} ‒ {"to":>{adjust}}: {"# of songs":>{adjust*2}}')
      
for name, group in grouped:
    print(f'{name - 10 ** precision:{adjust}} ‒ {name:{adjust}}: {len(group.groupby("name").groups):{adjust*2}}')
    stats.append({'from': name - 10 ** precision, 'to': name, '# of songs': len(group.groupby("name").groups)})
          
# print in your favorite way
# pd.DataFrame(stats, columns=['from', 'to', '# of songs'])

  from ‒     to:   # of songs
     0 ‒    100:            7
   100 ‒    200:           27
   200 ‒    300:          105
   300 ‒    400:          245
   400 ‒    500:          310
   500 ‒    600:          261
   600 ‒    700:          187
   700 ‒    800:          125
   800 ‒    900:           73
   900 ‒   1000:           49
  1000 ‒   1100:           31
  1100 ‒   1200:           16
  1200 ‒   1300:            4
  1300 ‒   1400:            6
  1400 ‒   1500:            5
  1500 ‒   1600:            3
  1600 ‒   1700:            1
  1700 ‒   1800:            1
  2200 ‒   2300:            1


In [ ]:
def train_generator(df, X_cols, y_cols, verbose=True):
    grouped = X.groupby(get_mask(df), level=0)
    
#     p = Pool(2)  # slowe because of memory
#     batches = p.starmap(create_batch, [(group, ceil_len) for ceil_len, group in grouped])
#     grouped = list(grouped)[:2]
    batches = [create_batch(group, ceil_len, verbose) for ceil_len, group in grouped]
    batches = [batch for batch in batches if len(batch.groupby('name').groups) > 8]
    
    while True:
        for batch in batches:
            yield [list2numpy(batch, col) for col in X_cols],\
                  [list2numpy(batch, col) for col in y_cols]

# test generated shapes
generator = train_generator(X, X_cols, y_cols)
for x, y in generator:
    print(f'x.shapes {[np.array(x_t).shape for x_t in x]}')
    print(f'y.shape {[np.array(y_t).shape for y_t in y]}\n')
    break

# Model

In [8]:
from tensorflow.keras.layers import Dense, LSTM, Flatten, Input, Activation, TimeDistributed, concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

In [26]:
def get_model(X, X_cols, y_cols):
    demo_row = X.iloc[0]
    X_shapes = [demo_row[col].shape[0] for col in X_cols]
    y_shapes = [demo_row[col].shape[0] for col in y_cols]
    
#     in1 = Input(shape=(None, 216)) # last blocks
#     in2 = Input(shape=(None, 2))   # time difference of previous and next beat
    inputs = [Input(shape=(None, shape)) for shape in X_shapes]
    
    time_dist = [TimeDistributed(Dense(shape, activation='sigmoid'))(inputs[i]) for i, shape in enumerate(X_shapes)]
#     out = time_dist
#     x1 = TimeDistributed(Dense(50, activation='elu'))(in1)
#     x2 = TimeDistributed(Dense(3, activation='elu'))(in2)
    
    out = concatenate(time_dist, axis=-1)
    out = LSTM(64, return_sequences=True)(out)
    out = LSTM(64, return_sequences=True)(out)
#     out = LSTM(64, return_sequences=True)(out)
#     out = LSTM(128, return_sequences=True)(out)
#     out = TimeDistributed(Dense(216, activation='sigmoid'))(out)
    outputs = [TimeDistributed(Dense(shape, activation='softmax'), name=col)(out) for shape, col in zip(y_shapes, y_cols)]

    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer='rmsprop',
#                   loss='binary_crossentropy',
                  loss='categorical_crossentropy',
#                   loss_weights=[1,2,3,1,2,3],
                  metrics=['accuracy'])
    
    return model

## Get a new model and show it

In [ ]:
model = get_model(X, X_cols, y_cols)
model.summary()

## Train model on generator

In [ ]:
def important_metric(metric_name):
    return 'val' in metric_name and 'acc' in metric_name
        

def train_on_generator(X, X_cols, y_cols, verbose_level=1, model=None, epochs=300):
    if not model:
        model = get_model(X, X_cols, y_cols)
    
    acc_results = {}
    stats_len = 30

    generator = train_generator(X, X_cols, y_cols, verbose_level>=2)
    for i, (x, y) in enumerate(generator):
        if i > epochs:
            break
        res = model.fit(x, y, batch_size=128, validation_split=0.1, verbose=verbose_level>=3)
        
        if verbose_level >= 1:
            if i % stats_len == 0:
                total_acc = (np.array(list(acc_results.values())) / stats_len).mean()
                display(Markdown(f'### Batch {i:4} | {total_acc:4.4}'))
                pprint([f'{key:30}: {val/stats_len}' for key, val in acc_results.items()])

                acc_results = {key: val[0] for key, val in res.history.items() if important_metric(key)}
            else:
                for key in acc_results:
                    acc_results[key] += res.history[key][0]
    
    
    acc_results = {key: val[0] for key, val in res.history.items() if important_metric(key)}
    total_acc = (np.array(list(acc_results.values()))).mean()
    display(Markdown(f'### Last epoch {i:4} results | {total_acc:4.4}'))
    pprint(acc_results)
    
    return model
    
# Test train_on_generator
# train_on_generator(X, X_cols, y_cols, 2, None, 2)

In [ ]:
# X.join(X.shift)
y_cols_shifted = [f'{x}_shifted' for x in y_cols]
shifted = X[y_cols].groupby('name').shift(1)
X[y_cols_shifted] = shifted
X = X.dropna()

In [ ]:
# Sanity check, try if the model performs well on _identity_
train_on_generator(X, y_cols, y_cols, verbose_level=1, model=None, epochs=300)

model = train_on_generator(X, ['times', 'mfcc'] + y_cols_shifted, y_cols, verbose_level=2, model=model, epochs=490)

### Empirical findings
- If the `y_cols_shifted` input is not provided, model tends to learn and stay on most common value of each classification.
- Sanity check porforms quickly over 90 % acc

### Improvements to be tested

1. More normalization of the data
    - Force the model to catch the underlying principle and not "mean"
1. Flip L / R hand and horizontal mirorring and rotations
1. Flip vertically with rotation
1. If one hand not used, mirror the other hand instead of "0"
1. Instead of generator, create snippets of 100 beats
    - Easier GPU training
    - Train it on gColab

## Hand evaluation
- Is needed since good results can be caused by a wrongly chosen matric!

In [ ]:
import copy

generator = train_generator(X, ['times', 'mfcc'] + y_cols_shifted, y_cols)
x, y = generator.__next__()
prediction = model.predict(x)

In [ ]:
f, t = 0, 20
for dim, (p, y_t) in enumerate(zip(prediction, y)):    
    df = pd.DataFrame(p[0][f:t])
    df = df.eq(df.where(df != 0).max(1), axis=0).astype(int)
    df.index.name = y_cols[dim]
    df_y = pd.DataFrame(y_t[0][f:t]).astype(int)
    df = df.join(df_y, rsuffix='_true')
    display(df)

In [ ]:
def eval_generator():
    while True:
        for x1, x2, y in zip(X['blocks'][300:], X['times'][300:], Y[300:]):
            yield [np.array([x1, ]), np.array([x2, ])], np.array([y, ])
            
model = get_model()
model.batch_size = 8            

model.fit_generator(train_generator(), steps_per_epoch=300, epochs=1, verbose=1, 
                    use_multiprocessing=False)
model.evaluate_generator(eval_generator(), steps=19, )

# Song snippets training

In [17]:
def generate_snippets(song_df, window=100, skip=50):
    stack = []
    ln = len(song_df)
    
    # Check if at least 1 window is possible
    if ln < window:
        return None
    
    # name information is contained in the grouping operation
    song_df = song_df.reset_index(level='name').drop(columns='name')

    for s in range(0, ln, skip):
        # Make sure the dataset contains ends of the songs
        if s + window > ln:
            stack.append(song_df.iloc[-window:])
        else:
            stack.append(song_df.iloc[s:s+window])
    
    df = pd.concat(stack, keys=list(range(0, len(song_df), skip)), names=['snippet', 'time'])
    return df
    
X = process_df(X, X_cols, y_cols)
X2 = X.iloc[:].groupby('name').apply(generate_snippets)


In [21]:
X_cols

['times',
 'mfcc',
 'r_dim0_shifted',
 'r_dim1_shifted',
 'r_dim2_shifted',
 'l_dim0_shifted',
 'l_dim1_shifted',
 'l_dim2_shifted']

In [22]:
def create_training_data(X, groupby=('name', 'snippet')):
    return [list2numpy(X, col, groupby) for col in X_cols],\
           [list2numpy(X, col, groupby) for col in y_cols]

x, y = create_training_data(X2)

In [34]:
X.to_pickle('X_gdrive.zip')
# X

times  \
name                                    time                                                     
../data/1My Truth/blocks/Hard.pkl       3.349315      [0.8219178082191778, 0.8219178082191778]   
                                        4.171233      [0.8219178082191778, 0.2054794520547949]   
                                        4.376712      [0.2054794520547949, 0.2054794520547949]   
                                        4.582192      [0.2054794520547949, 0.2054794520547949]   
                                        4.787671      [0.2054794520547949, 0.2054794520547949]   
                                        4.993151      [0.2054794520547949, 0.8219178082191778]   
                                        5.815068      [0.8219178082191778, 0.8219178082191769]   
                                        6.636986      [0.8219178082191769, 0.2054794520547949]   
                                        6.842466      [0.2054794520547949, 0.2054794520547949]   
                                        7.047945      [0.2054794520547949, 0.2054794520547949]   
                                        7.253425      [0.2054794520547949, 0.2054794520547949]   
                                        7.458904      [0.2054794520547949, 1.2328767123287676]   
                                        8.691781      [1.2328767123287676, 0.4109589041095898]   
                                        9.102740      [0.4109589041095898, 0.6164383561643838]   
                                        9.719178       [0.6164383561643838, 0.205479452054794]   
                                        9.924658       [0.205479452054794, 1.7054794520547958]   
                                        11.630137     [1.7054794520547958, 0.4232876712328757]   
                                        12.053425    [0.4232876712328757, 0.38630136986301444]   
                                        12.439726    [0.38630136986301444, 0.7849315068493148]   
                                        13.224658     [0.7849315068493148, 0.8095890410958901]   
                                        14.034247      [0.8095890410958901, 0.205479452054794]   
                                        14.239726       [0.205479452054794, 0.205479452054794]   
                                        14.445205       [0.205479452054794, 0.205479452054794]   
                                        14.650685       [0.205479452054794, 0.205479452054794]   
                                        14.856164      [0.205479452054794, 0.8712328767123303]   
                                        15.727397     [0.8712328767123303, 0.7849315068493148]   
                                        16.512329    [0.7849315068493148, 0.20547945205479579]   
                                        16.717808   [0.20547945205479579, 0.23013698630137114]   
                                        16.947945    [0.23013698630137114, 0.1726027397260239]   
                                        17.120548    [0.1726027397260239, 0.20547945205479579]   
...                                                                                        ...   
../data/Torture Dance/blocks/Expert.pkl 131.164486    [0.5607476635514104, 0.2803738317757052]   
                                        131.444860    [0.2803738317757052, 0.2803738317757052]   
                                        131.725234    [0.2803738317757052, 0.2803738317757052]   
                                        132.005607    [0.2803738317757052, 0.2803738317757052]   
                                        132.285981    [0.2803738317757052, 0.5607476635514104]   
                                        132.846729    [0.5607476635514104, 0.5607476635514104]   
                                        133.407477    [0.5607476635514104, 0.2803738317757052]   
                                        133.687850    [0.2803738317757052, 0.2803738317757052]   
                                        133.968224    [0.28037383177570

In [23]:
model = get_model(X, X_cols, y_cols)

# model.fit(x, y, batch_size=32, verbose=True)

In [27]:
res = model.fit(x, y, batch_size=128, epochs=10, validation_split=0.1, verbose=True)

Train on 15109 samples, validate on 1679 samples
Epoch 1/10
15109/15109 [==============================] - 11s 711us/sample - loss: 6.7452 - r_dim0_loss: 0.7029 - r_dim1_loss: 1.2290 - r_dim2_loss: 1.4358 - l_dim0_loss: 0.7016 - l_dim1_loss: 1.2408 - l_dim2_loss: 1.4350 - r_dim0_acc: 0.7185 - r_dim1_acc: 0.4405 - r_dim2_acc: 0.4964 - l_dim0_acc: 0.7200 - l_dim1_acc: 0.4303 - l_dim2_acc: 0.4986 - val_loss: 6.9621 - val_r_dim0_loss: 0.7315 - val_r_dim1_loss: 1.2666 - val_r_dim2_loss: 1.4823 - val_l_dim0_loss: 0.7335 - val_l_dim1_loss: 1.2682 - val_l_dim2_loss: 1.4799 - val_r_dim0_acc: 0.7230 - val_r_dim1_acc: 0.4186 - val_r_dim2_acc: 0.4855 - val_l_dim0_acc: 0.7254 - val_l_dim1_acc: 0.4078 - val_l_dim2_acc: 0.4875
Epoch 2/10
15109/15109 [==============================] - 11s 733us/sample - loss: 6.7058 - r_dim0_loss: 0.6997 - r_dim1_loss: 1.2243 - r_dim2_loss: 1.4245 - l_dim0_loss: 0.6986 - l_dim1_loss: 1.2352 - l_dim2_loss: 1.4235 - r_dim0_acc: 0.7186 - r_dim1_acc: 0.4444 - r_dim2_acc: 

In [28]:
res = model.fit(x, y, batch_size=128, epochs=20, validation_split=0.1, verbose=True)

Train on 15109 samples, validate on 1679 samples
Epoch 1/20
15109/15109 [==============================] - 12s 777us/sample - loss: 6.4416 - r_dim0_loss: 0.6761 - r_dim1_loss: 1.1911 - r_dim2_loss: 1.3487 - l_dim0_loss: 0.6750 - l_dim1_loss: 1.2036 - l_dim2_loss: 1.3471 - r_dim0_acc: 0.7211 - r_dim1_acc: 0.4670 - r_dim2_acc: 0.5312 - l_dim0_acc: 0.7233 - l_dim1_acc: 0.4582 - l_dim2_acc: 0.5336 - val_loss: 6.5635 - val_r_dim0_loss: 0.6937 - val_r_dim1_loss: 1.2127 - val_r_dim2_loss: 1.3749 - val_l_dim0_loss: 0.6888 - val_l_dim1_loss: 1.2255 - val_l_dim2_loss: 1.3678 - val_r_dim0_acc: 0.7240 - val_r_dim1_acc: 0.4450 - val_r_dim2_acc: 0.5272 - val_l_dim0_acc: 0.7271 - val_l_dim1_acc: 0.4381 - val_l_dim2_acc: 0.5330
Epoch 2/20
15109/15109 [==============================] - 12s 779us/sample - loss: 6.4158 - r_dim0_loss: 0.6746 - r_dim1_loss: 1.1873 - r_dim2_loss: 1.3409 - l_dim0_loss: 0.6733 - l_dim1_loss: 1.2004 - l_dim2_loss: 1.3393 - r_dim0_acc: 0.7216 - r_dim1_acc: 0.4694 - r_dim2_acc: 

In [29]:
res = model.fit(x, y, batch_size=128, epochs=60, validation_split=0.1, verbose=True)

Train on 15109 samples, validate on 1679 samples
Epoch 1/60
15109/15109 [==============================] - 12s 816us/sample - loss: 6.0495 - r_dim0_loss: 0.6431 - r_dim1_loss: 1.1284 - r_dim2_loss: 1.2441 - l_dim0_loss: 0.6423 - l_dim1_loss: 1.1453 - l_dim2_loss: 1.2464 - r_dim0_acc: 0.7356 - r_dim1_acc: 0.5017 - r_dim2_acc: 0.5778 - l_dim0_acc: 0.7365 - l_dim1_acc: 0.4924 - l_dim2_acc: 0.5784 - val_loss: 6.1323 - val_r_dim0_loss: 0.6407 - val_r_dim1_loss: 1.1556 - val_r_dim2_loss: 1.2585 - val_l_dim0_loss: 0.6398 - val_l_dim1_loss: 1.1770 - val_l_dim2_loss: 1.2608 - val_r_dim0_acc: 0.7348 - val_r_dim1_acc: 0.4791 - val_r_dim2_acc: 0.5749 - val_l_dim0_acc: 0.7376 - val_l_dim1_acc: 0.4681 - val_l_dim2_acc: 0.5775
Epoch 2/60
15109/15109 [==============================] - 12s 811us/sample - loss: 6.0374 - r_dim0_loss: 0.6420 - r_dim1_loss: 1.1264 - r_dim2_loss: 1.2410 - l_dim0_loss: 0.6412 - l_dim1_loss: 1.1434 - l_dim2_loss: 1.2434 - r_dim0_acc: 0.7361 - r_dim1_acc: 0.5028 - r_dim2_acc: 

15109/15109 [==============================] - 12s 823us/sample - loss: 5.8407 - r_dim0_loss: 0.6256 - r_dim1_loss: 1.0899 - r_dim2_loss: 1.1948 - l_dim0_loss: 0.6255 - l_dim1_loss: 1.1084 - l_dim2_loss: 1.1966 - r_dim0_acc: 0.7431 - r_dim1_acc: 0.5244 - r_dim2_acc: 0.5959 - l_dim0_acc: 0.7435 - l_dim1_acc: 0.5132 - l_dim2_acc: 0.5969 - val_loss: 5.9461 - val_r_dim0_loss: 0.6254 - val_r_dim1_loss: 1.1183 - val_r_dim2_loss: 1.2183 - val_l_dim0_loss: 0.6222 - val_l_dim1_loss: 1.1433 - val_l_dim2_loss: 1.2187 - val_r_dim0_acc: 0.7412 - val_r_dim1_acc: 0.5008 - val_r_dim2_acc: 0.5872 - val_l_dim0_acc: 0.7437 - val_l_dim1_acc: 0.4889 - val_l_dim2_acc: 0.5898
Epoch 26/60
15109/15109 [==============================] - 12s 815us/sample - loss: 5.8347 - r_dim0_loss: 0.6252 - r_dim1_loss: 1.0888 - r_dim2_loss: 1.1934 - l_dim0_loss: 0.6250 - l_dim1_loss: 1.1074 - l_dim2_loss: 1.1951 - r_dim0_acc: 0.7434 - r_dim1_acc: 0.5248 - r_dim2_acc: 0.5966 - l_dim0_acc: 0.7437 - l_dim1_acc: 0.5135 - l_dim2_a

15109/15109 [==============================] - 12s 806us/sample - loss: 5.7377 - r_dim0_loss: 0.6161 - r_dim1_loss: 1.0725 - r_dim2_loss: 1.1707 - l_dim0_loss: 0.6155 - l_dim1_loss: 1.0905 - l_dim2_loss: 1.1723 - r_dim0_acc: 0.7468 - r_dim1_acc: 0.5340 - r_dim2_acc: 0.6037 - l_dim0_acc: 0.7474 - l_dim1_acc: 0.5232 - l_dim2_acc: 0.6043 - val_loss: 5.8968 - val_r_dim0_loss: 0.6250 - val_r_dim1_loss: 1.1115 - val_r_dim2_loss: 1.1998 - val_l_dim0_loss: 0.6223 - val_l_dim1_loss: 1.1380 - val_l_dim2_loss: 1.2002 - val_r_dim0_acc: 0.7412 - val_r_dim1_acc: 0.5084 - val_r_dim2_acc: 0.5924 - val_l_dim0_acc: 0.7440 - val_l_dim1_acc: 0.4936 - val_l_dim2_acc: 0.5951
Epoch 50/60
15109/15109 [==============================] - 12s 808us/sample - loss: 5.7345 - r_dim0_loss: 0.6160 - r_dim1_loss: 1.0717 - r_dim2_loss: 1.1699 - l_dim0_loss: 0.6154 - l_dim1_loss: 1.0898 - l_dim2_loss: 1.1716 - r_dim0_acc: 0.7469 - r_dim1_acc: 0.5342 - r_dim2_acc: 0.6038 - l_dim0_acc: 0.7474 - l_dim1_acc: 0.5234 - l_dim2_a